# TensorFlow2教程-使用keras训练模型

本指南包含了TensorFlow 2.0中在以下两种情况下的训练，评估和预测（推理）模型：

+ 使用内置的训练和评估API（例如model.fit()，model.evaluate()，model.predict()）。
+ 使用eager execution 和GradientTape对象从头开始编写自定义循环。

无论是使用内置循环还是编写自己的循环，模型和评估训练在每种Keras模型中严格按照相同的方式工作，无论是Sequential 模型, 函数式 API, 还是模型子类化。



In [2]:
from __future__ import absolute_import, division, print_function
import tensorflow as tf
tf.keras.backend.clear_session()
import tensorflow.keras as keras
import tensorflow.keras.layers as layers

## 1 一般的模型构造、训练、测试流程


使用内置的训练和评估API对模型进行训练和验证。



In [3]:
# 模型构造
inputs = keras.Input(shape=(784,), name='mnist_input')
h1 = layers.Dense(64, activation='relu')(inputs)
h1 = layers.Dense(64, activation='relu')(h1)
outputs = layers.Dense(10, activation='softmax')(h1)
model = keras.Model(inputs, outputs)
# keras.utils.plot_model(model, 'net001.png', show_shapes=True)

model.compile(optimizer=keras.optimizers.RMSprop(),
             loss=keras.losses.SparseCategoricalCrossentropy(),
             metrics=[keras.metrics.SparseCategoricalAccuracy()])

端到端的模型训练。



In [4]:
# 载入数据
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784).astype('float32') /255
x_test = x_test.reshape(10000, 784).astype('float32') /255

# 保证还是float 32？ 否则后面会出现：TypeError: Input 'y' of 'Sub' Op has type float32 that does not match type uint8 of argument 'x'.
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')



# 取验证数据
x_val = x_train[-10000:]
y_val = y_train[-10000:]

x_train = x_train[:-10000]
y_train = y_train[:-10000]

# 训练模型
history = model.fit(x_train, y_train, batch_size=64, epochs=3,
         validation_data=(x_val, y_val))
print('history:')
print(history.history)

result = model.evaluate(x_test, y_test, batch_size=128)
print('evaluate:')
print(result)
pred = model.predict(x_test[:2])
print('predict:')
print(pred)


11493376/11490434 [==============================] - 10s 1us/step
Epoch 1/3
782/782 [==============================] - 1s 1ms/step - loss: 0.3342 - sparse_categorical_accuracy: 0.9048 - val_loss: 0.1805 - val_sparse_categorical_accuracy: 0.9461
Epoch 2/3
782/782 [==============================] - 1s 991us/step - loss: 0.1564 - sparse_categorical_accuracy: 0.9535 - val_loss: 0.1324 - val_sparse_categorical_accuracy: 0.9624
Epoch 3/3
782/782 [==============================] - 1s 983us/step - loss: 0.1146 - sparse_categorical_accuracy: 0.9651 - val_loss: 0.1195 - val_sparse_categorical_accuracy: 0.9653
history:
{'loss': [0.33420199155807495, 0.15635664761066437, 0.11455204337835312], 'sparse_categorical_accuracy': [0.9047799706459045, 0.953499972820282, 0.9651399850845337], 'val_loss': [0.18046434223651886, 0.13240696489810944, 0.11946222186088562], 'val_sparse_categorical_accuracy': [0.9460999965667725, 0.9624000191688538, 0.9653000235557556]}
79/79 [==============================] - 0s 